# Spark Structured Streaming with Kafka

In [1]:
# spark.conf.set("spark.sql.session.timeZone", "UTC")

## Preparing DataFrame

In [2]:
d = spark.readStream.format('kafka'). \
    option('kafka.bootstrap.servers', 'master:9092,slave01:9092,slave02:9092,slave03:9092,slave04:9092,slave05:9092'). \
    option('subscribe', 'gios'). \
    option('startingOffsets', 'earliest'). \
    load()

In [3]:
q = d.selectExpr('CAST(key AS STRING)', 'CAST(value AS STRING)', 'partition', 'offset', 'timestamp')

### Additional attributes

In [4]:
from pyspark.sql.types import DoubleType

qWithValueAsDouble = q.withColumn('value_as_double', q['value'].cast(DoubleType()))

#### Parsing Kafka's timestamp

In [5]:
from pyspark.sql import functions as F

qWithTs = qWithValueAsDouble.withColumn('day', qWithValueAsDouble['timestamp'].substr(1, 10)). \
            withColumn('parsed_timestamp', F.to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss'))

In [6]:
qWithDD = qWithTs.withColumn('day_diff', F.datediff(F.current_timestamp(), qWithTs['parsed_timestamp']))

## Analytics

### Filtering (getting last 5 _full_ days)

In [7]:
filtered = qWithDD.filter((qWithDD['day_diff'] > 0) & (qWithDD['day_diff'] < 6))

### Grouping by station name and day

In [8]:
grouped = filtered.groupBy('key', 'day').avg('value_as_double').withColumnRenamed('avg(value_as_double)', 'avg_NO2')

### Writing stream

In [9]:
grouped.writeStream.format('memory').queryName('in_memory').outputMode('complete').start()

## Results

In [12]:
spark.sql('SELECT t.rank, t.key, t.day, t.avg_NO2 FROM (SELECT key, day, avg_NO2, DENSE_RANK() OVER (PARTITION BY key ORDER BY avg_NO2 DESC) as rank FROM in_memory) as t WHERE t.rank < 3 ORDER BY t.key ASC, t.avg_NO2 DESC').show()

+----+--------------------+----------+------------------+
|rank|                 key|       day|           avg_NO2|
+----+--------------------+----------+------------------+
|   1| KMŚ Puszcza Borecka|2020-05-19|0.5323618181818182|
|   2| KMŚ Puszcza Borecka|2020-05-18|0.4100247826086957|
|   1|WIOŚ Biskupiec-Mo...|2020-05-20|25.828500000000002|
|   2|WIOŚ Biskupiec-Mo...|2020-05-22| 25.45395217391304|
|   1|WIOŚ Elbląg ul. B...|2020-05-19| 9.562459565217392|
|   2|WIOŚ Elbląg ul. B...|2020-05-18| 7.971984347826088|
|   1|            WIOŚ Ełk|2020-05-19| 6.003737826086956|
|   2|            WIOŚ Ełk|2020-05-22| 4.208602173913043|
|   1|WIOŚ Gołdap ul. J...|2020-05-19|  6.43403695652174|
|   2|WIOŚ Gołdap ul. J...|2020-05-22| 5.944795652173913|
|   1|WIOŚ Olsztyn ul. ...|2020-05-18| 5.636427826086957|
|   2|WIOŚ Olsztyn ul. ...|2020-05-19| 4.982760434782608|
|   1|WIOŚ Ostróda Piłs...|2020-05-18| 7.895448695652173|
|   2|WIOŚ Ostróda Piłs...|2020-05-22| 7.210130434782608|
+----+--------

In [11]:
# spark.stop()